## Corpus testing

This notebook takes works with known labels and looks how good it is attributed to its author. 

In [1]:
import pandas as pd
import numpy as np

import re

from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level="INFO")
logger = logging.getLogger("ruzicka")

from ruzicka.BDIVerifier import BDIVerifier

In [13]:
# load preprocessed data

corpus = pd.read_csv('03_tests/test_1000_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,la,et,les,vous,le,...,cela,beaucoup,voir,alors,ici,enfin,voit,rendre,cependant,seulement
0,Avis,Baudeau,0,0__Baudeau_Avis,3.7,1.7,2.9,2.7,0.2,2.0,...,0.2,0.1,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0
1,Explication,Baudeau,0,0__Baudeau_Explication,4.8,3.9,3.1,3.9,0.9,1.0,...,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.0
2,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration,5.6,2.5,1.7,1.4,0.0,2.3,...,0.0,0.2,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.1
3,Idees sur les besoins 1,Baudeau,0,0__Baudeau_Idees sur les besoins 1,4.1,2.5,1.8,2.7,0.2,2.6,...,0.0,0.0,0.1,0.0,0.2,0.0,0.0,0.0,0.0,0.0
4,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2,4.0,2.5,2.3,3.0,0.1,2.5,...,0.0,0.0,0.1,0.0,0.2,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5403,elements T1,Marmontel,99,99__Marmontel_elements T1,6.4,3.2,3.1,2.1,0.1,1.8,...,0.2,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.1
5404,elements T2,Marmontel,99,99__Marmontel_elements T2,4.9,3.4,3.8,2.3,0.0,1.7,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.3
5405,elements T3,Marmontel,99,99__Marmontel_elements T3,5.3,2.8,4.2,0.4,0.0,2.7,...,0.0,0.0,0.0,0.2,0.0,0.0,0.1,0.0,0.0,0.1
5406,Manuel d-epictete,Naigeon,99,99__Naigeon_Manuel d-epictete,4.2,3.5,3.0,1.0,0.0,1.8,...,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1


In [14]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Explication,Baudeau,0,0__Baudeau_Explication
2,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration
3,Idees sur les besoins 1,Baudeau,0,0__Baudeau_Idees sur les besoins 1
4,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2
...,...,...,...,...
5403,elements T1,Marmontel,99,99__Marmontel_elements T1
5404,elements T2,Marmontel,99,99__Marmontel_elements T2
5405,elements T3,Marmontel,99,99__Marmontel_elements T3
5406,Manuel d-epictete,Naigeon,99,99__Naigeon_Manuel d-epictete


In [15]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(5408, 200)

In [16]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Explication,0,Baudeau,0,0__Baudeau_Explication
2,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
3,Idees sur les besoins 1,0,Baudeau,0,0__Baudeau_Idees sur les besoins 1
4,Idees sur les besoins 2,0,Baudeau,0,0__Baudeau_Idees sur les besoins 2
...,...,...,...,...,...
5403,elements T1,10,Marmontel,99,99__Marmontel_elements T1
5404,elements T2,10,Marmontel,99,99__Marmontel_elements T2
5405,elements T3,10,Marmontel,99,99__Marmontel_elements T3
5406,Manuel d-epictete,13,Naigeon,99,99__Naigeon_Manuel d-epictete


In [17]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [18]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [80]:
# Loop that takes a work from each author
# and looks whether its (known) author is a good much according to BDI;
# nb: not getting results for a work vs all authors
# (writing results in a .csv, visualisation done later with R)

# select an author, look for his unique works
for a in label_uniques.values:
    #print(author)
    
    # take a subset with only works by an author, find unique works
    unique_works = raw_df[raw_df.author == a]['work'].unique()
    # unique_works is a numpy array of works of each author
    
    for w in unique_works:
        print(f"\n\n#######################\n{a}: {w}\n")
        
        # select each work as a problem set (problems-metadata, problems_X-word freqs)
        problems = raw_df[raw_df.work == w].reset_index(drop=True).copy()
        problems_X = X[raw_df.work == w].reset_index(drop=True).copy()
        print(problems.iloc[1:2,:])
        
        # extract the rest of the corpus
        rest = raw_df[raw_df.work != w].reset_index(drop=True).copy()
        rest_X = X[raw_df.work != w].reset_index(drop=True).copy()
        #print(f"\nRest of the corpus:\n{rest.iloc[1:6,:]}")
        
        # scaling
        ss = StandardScaler(with_mean=False).fit(rest_X)
        rest_scaled_X = ss.transform(rest_X)
        problems_scaled_X = ss.transform(problems_X)
        
        #print(problems_scaled_X)
        
        # fit 
        bdi_mm.fit(rest_scaled_X, rest.author_label)
        
        # prediction
        bdi_mm.predict_proba(
            problems_scaled_X, 
            # get_loc - select the author of the work (? is that right??)
            [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
        )
        
        # output results
        res = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
        
        # writing
        fh = '03_tests/tests/ch1000/mfw200/'+a+'_'+w+'.csv'
        res.to_csv(fh)
        print('Results written in:',fh)
        
        rest_scaled_X = None
        res = None

12/19/2024 05:21:18 [ruzicka:INFO] Fitting on 5358 documents...
12/19/2024 05:21:18 [ruzicka:INFO] Predicting on 50 documents




#######################
Baudeau: Avis 

    work  author_label   author  chunk_num               tag
1  Avis              0  Baudeau          1  1__Baudeau_Avis 


12/19/2024 05:21:31 [ruzicka:INFO] Fitting on 5386 documents...
12/19/2024 05:21:31 [ruzicka:INFO] Predicting on 22 documents


Results written in: 03_tests/tests/ch1000/mfw200/Baudeau_Avis .csv


#######################
Baudeau: Explication

          work  author_label   author  chunk_num                     tag
1  Explication             0  Baudeau          1  1__Baudeau_Explication


12/19/2024 05:21:38 [ruzicka:INFO] Fitting on 5390 documents...
12/19/2024 05:21:38 [ruzicka:INFO] Predicting on 18 documents


...

Results written in: 03_tests/tests/ch1000/mfw200/Saint-Lambert_Oeuvres Phil T2.csv


#######################
Saint-Lambert: Oeuvres Phil T3

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T3            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T3  


12/19/2024 05:53:33 [ruzicka:INFO] Fitting on 5318 documents...
12/19/2024 05:53:33 [ruzicka:INFO] Predicting on 90 documents


Results written in: 03_tests/tests/ch1000/mfw200/Saint-Lambert_Oeuvres Phil T3.csv


#######################
Saint-Lambert: Oeuvres Phil T4

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T4            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T4  


12/19/2024 05:54:03 [ruzicka:INFO] Fitting on 5363 documents...
12/19/2024 05:54:03 [ruzicka:INFO] Predicting on 45 documents


Results written in: 03_tests/tests/ch1000/mfw200/Saint-Lambert_Oeuvres Phil T4.csv


#######################
Saint-Lambert: Oeuvres Phil T5

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T5            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T5  
Results written in: 03_tests/tests/ch1000/mfw200/Saint-Lambert_Oeuvres Phil T5.csv


# FP1

Test FP1 chunks with BDI

In [91]:
# load preprocessed data

corpus = pd.read_csv('03_tests/FP1_2000_100mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,des,et,la,les,vous,...,toujours,après,cet,autre,donc,jamais,encore,avoir,ainsi,peu
0,Avis,Baudeau,0,0__Baudeau_Avis,3.45,2.20,2.50,1.40,2.35,0.65,...,0.10,0.00,0.05,0.05,0.40,0.00,0.00,0.30,0.00,0.05
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.85,2.80,2.60,2.40,1.65,...,0.10,0.10,0.05,0.15,0.10,0.10,0.05,0.15,0.05,0.05
2,Explication,Baudeau,0,0__Baudeau_Explication,4.90,2.60,3.80,3.10,4.65,0.95,...,0.15,0.10,0.00,0.00,0.10,0.00,0.10,0.10,0.05,0.20
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration,5.60,1.90,2.20,3.65,1.85,0.10,...,0.15,0.00,0.10,0.00,0.10,0.25,0.10,0.20,0.05,0.00
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance,4.90,1.85,2.50,2.35,2.75,0.10,...,0.20,0.05,0.05,0.10,0.15,0.15,0.10,0.10,0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4422,elements T3,Marmontel,96,96__Marmontel_elements T3,4.15,1.20,3.35,3.00,1.15,0.00,...,0.00,0.00,0.15,0.20,0.00,0.05,0.05,0.20,0.00,0.00
4423,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19,5.25,1.85,0.75,3.75,2.65,0.00,...,0.05,0.15,0.05,0.15,0.00,0.00,0.05,0.15,0.05,0.20
4424,elements T3,Marmontel,97,97__Marmontel_elements T3,4.60,0.95,3.10,3.40,1.55,0.00,...,0.00,0.05,0.00,0.30,0.05,0.15,0.05,0.10,0.10,0.00
4425,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19,4.90,2.95,0.75,3.25,3.20,0.05,...,0.00,0.05,0.05,0.15,0.05,0.00,0.15,0.20,0.05,0.20


In [92]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...
4422,elements T3,Marmontel,96,96__Marmontel_elements T3
4423,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19
4424,elements T3,Marmontel,97,97__Marmontel_elements T3
4425,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19


In [93]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(4427, 100)

In [94]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4422,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4423,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4424,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4425,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


In [95]:
# select an "unknown" work

problems = raw_df[raw_df.work == "FP1"].reset_index(drop=True).copy()
problems_X = X[raw_df.work == "FP1"].reset_index(drop=True).copy()
problems

,work,author_label,author,chunk_num,tag
0,FP1,7,HDI,0,0__HDI_FP1
1,FP1,7,HDI,1,1__HDI_FP1
2,FP1,7,HDI,2,2__HDI_FP1


In [96]:
# extract the rest of the corpus

rest = raw_df[raw_df.work != "FP1"].reset_index(drop=True).copy()
rest_X = X[raw_df.work != "FP1"].reset_index(drop = True).copy()
rest

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4419,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4420,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4421,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4422,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


**Scaling**

In [97]:
ss = StandardScaler(with_mean=False).fit(rest_X)

In [98]:
rest_scaled_X = ss.transform(rest_X)
problems_scaled_X = ss.transform(problems_X)

**Verification**

In [99]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [100]:
# fit 
bdi_mm.fit(rest_scaled_X, rest.author_label)

01/21/2025 05:16:38 [ruzicka:INFO] Fitting on 4424 documents...


In [101]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'HDI', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [102]:
for label in label_uniques.values:
    print(f"Testing against {label}")
    code = label_uniques.get_loc(label)
    print(
        f"Bootstrap Match Strength (one per chunk, 0-1.0): {bdi_mm.predict_proba(problems_scaled_X, [code] * problems_scaled_X.shape[0])}"
    )

01/21/2025 05:16:43 [ruzicka:INFO] Predicting on 3 documents


Testing against Baudeau


01/21/2025 05:16:43 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.775 0.63  0.674]
Testing against Chastellux


01/21/2025 05:16:44 [ruzicka:INFO] Predicting on 3 documents


...

Bootstrap Match Strength (one per chunk, 0-1.0): [0.21  0.414 0.73 ]
Testing against Raynal


01/21/2025 05:16:55 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.685 0.574 0.6  ]
Testing against Rivière


01/21/2025 05:16:56 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.498 0.534 0.514]
Testing against Saint-Lambert
Bootstrap Match Strength (one per chunk, 0-1.0): [0.574 0.944 0.947]


In [103]:
authors = label_uniques.values

fh = ''

for a in authors:

    print(a)
    
    bdi_mm.predict_proba(
        problems_scaled_X, [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
    )

    
    x = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
    
    fh = '03_tests/tests/fp1/fp1_vs_' + a + '.csv'

    x.to_csv(fh)

01/21/2025 05:16:58 [ruzicka:INFO] Predicting on 3 documents


Baudeau


01/21/2025 05:16:58 [ruzicka:INFO] Predicting on 3 documents


Chastellux


01/21/2025 05:16:59 [ruzicka:INFO] Predicting on 3 documents


...

Raynal


01/21/2025 05:17:10 [ruzicka:INFO] Predicting on 3 documents


Rivière


01/21/2025 05:17:11 [ruzicka:INFO] Predicting on 3 documents


Saint-Lambert
